In [1]:
import sys
sys.path.append('src')

from data.data_loader import DataLoader
from features.feature_engineering import FeatureEngineering
from models.model_evaluator import ModelEvaluator

import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

## 1. 数据加载

加载Excel数据并划分训练集和测试集。

In [2]:
# 初始化数据加载器
file_path = r"./data/raw/training_data.xlsx"  # 请根据实际数据文件路径调整
data_loader = DataLoader(file_path)

# 加载数据
df = data_loader.load_data(
    sheet_name='msw_result',
    target_column='MSW',
    feature_columns=['Population', 'GDP PPP 2017', 'GDP PPP/capita 2017']
)

In [3]:
# 划分国家外部测试集
train_data, country_test_data = data_loader.split_data_by_countries(df,test_size= 0.15,random_state=123)
# 划分时间外部测试集
train_data, time_test_data = data_loader.split_data_by_time(train_data)
# 数据检查
data_loader.analyze_datasets(df)
data_loader.analyze_datasets(train_data)
data_loader.analyze_datasets(country_test_data)
data_loader.analyze_datasets(time_test_data)

train_data.to_csv('src/data/train_data.csv',index=False)
country_test_data.to_csv('src/data/country_test_data.csv',index=False)
time_test_data.to_csv('src/data/time_test_data.csv',index=False)



数据集统计信息:
总数据条数: 1982
国家总数: 69
包含的国家: Albania, Algeria, Argentina, Australia, Austria, Bangladesh, Belgium, Bosnia and Herzegovina, Brazil, Bulgaria, Canada, China, Colombia, Congo, Dem. Rep., Croatia, Cyprus, Czechia, Denmark, Egypt, Arab Rep., Estonia, Ethiopia, Finland, France, Germany, Greece, Hungary, Iceland, India, Indonesia, Iran, Islamic Rep., Iraq, Ireland, Italy, Japan, Korea, Rep., Latvia, Lithuania, Luxembourg, Malaysia, Malta, Mexico, Montenegro, Morocco, Netherlands, Nigeria, North Macedonia, Norway, Pakistan, Peru, Philippines, Poland, Portugal, Romania, Russian Federation, Saudi Arabia, Serbia, Slovak Republic, Slovenia, South Africa, Spain, Sweden, Switzerland, Tanzania, Thailand, Turkiye, Uganda, Ukraine, United Kingdom, United States

数据集统计信息:
总数据条数: 1476
国家总数: 59
包含的国家: Albania, Argentina, Australia, Austria, Bangladesh, Belgium, Bosnia and Herzegovina, Bulgaria, Canada, China, Colombia, Congo, Dem. Rep., Croatia, Cyprus, Denmark, Egypt, Arab Rep., Ethiopia, Finlan

## 2. 特征工程

创建时间特征和类别交互特征。

In [ ]:
# 初始化特征工程
feature_engineering = FeatureEngineering()

# 处理训练集特征
train_data_processed, target_column = feature_engineering.fit_transform(
    train_data,
    target_column='MSW',
    categorical_columns=['Region', 'Income Group']
)

# 处理测试集特征
country_test_data_processed, _ = feature_engineering.transform(
    country_test_data,
    target_column='MSW',
)

# 处理测试集特征
time_test_data_processed, _ = feature_engineering.transform(
    time_test_data,
    target_column='MSW'
)

## 3. 模型训练和评估

训练多个模型并评估它们的性能。

In [5]:
# 初始化模型评估器
model_evaluator = ModelEvaluator()

# 设置实验
feature_cols = [col for col in train_data_processed.columns 
               if col not in ['MSW', 'Country Name', 'Year']]

train = train_data_processed[feature_cols]

In [6]:
model_evaluator.setup_experiment(
    train_size = 0.8,
    train_data=train,
    target_column=target_column,
    categorical_features=['Region', 'Income Group'],
)

,Description,Value
0,Session id,866
1,Target,MSW_None
2,Target type,Regression
3,Original data shape,"(1476, 23)"
4,Transformed data shape,"(1476, 32)"
5,Transformed train set shape,"(1180, 32)"
6,Transformed test set shape,"(296, 32)"
7,Numeric features,20
8,Categorical features,2
9,Preprocess,True


In [7]:
# 训练模型
models = model_evaluator.train_top_models(n_models=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,10433600.5889,812123826342702.7500,20095664.4396,0.6668,0.8075,1.4835,0.0340
gbr,Gradient Boosting Regressor,10210872.5063,768041760913811.0000,20379628.3067,0.6182,0.6997,0.8086,0.0420
catboost,CatBoost Regressor,10331231.2455,1027153043303803.7500,22623511.5519,0.6114,0.6846,0.8071,0.3300
ada,AdaBoost Regressor,11409574.5330,750101538061141.7500,20736692.3738,0.5955,1.1594,3.0127,0.0220
rf,Random Forest Regressor,11162658.9722,796650210101648.2500,21542654.3273,0.5353,0.7209,0.9613,0.0460
dt,Decision Tree Regressor,11570815.0209,804359922549666.1250,22195458.3902,0.4783,0.8343,1.3692,0.0140
lightgbm,Light Gradient Boosting Machine,11077607.4802,888794293528288.1250,23701961.5880,0.4638,0.7520,1.3391,0.0860
xgboost,Extreme Gradient Boosting,12158289.6500,1015842111855001.6250,25139749.5000,0.4305,0.8178,1.3229,0.0580
ridge,Ridge Regression,13840612.2218,807854953513190.6250,24402466.6836,0.3082,1.2726,3.8683,0.4220
omp,Orthogonal Matching Pursuit,14691545.9960,875536813651081.5000,24689124.6260,0.2357,1.5539,4.9995,0.3540


In [8]:
# 准备测试数据
X_test = country_test_data_processed[feature_cols]
y_test = country_test_data_processed[target_column]

# 评估所有模型组合
results_country = model_evaluator.evaluate_ensemble_combinations(X_test, y_test)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,5062041.9190,64253999093205.8125,8015859.2236,0.8118,0.4277,0.4380


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,3442371.3735,33468606910407.1797,5785205.8659,0.9020,0.3274,0.3162


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,4613232.0859,60109691459546.9688,7753044.0125,0.8239,0.4574,0.4627


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,21300693.8622,2670682209187896.0000,51678643.6469,-0.0485,1.1743,1.0890
1,15356214.0066,984396030609005.0000,31375086.1450,0.7477,0.6219,0.6507
2,2898821.0493,18997611881735.1133,4358624.9990,0.9454,0.2403,0.1867
3,2807112.5457,15523403636233.3672,3939975.0807,0.8903,1.2025,3.2951
4,3136282.9548,21591925295527.5547,4646711.2344,0.8656,0.3200,0.2802
Mean,9099824.8837,742238236122079.3750,19199808.2212,0.6801,0.7118,1.1003
Std,7766828.4151,1034220393175367.2500,19328879.9570,0.3700,0.4095,1.1425


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,4200865.3757,46641206781928.2578,6829436.7837,0.8634,0.3686,0.3612


...

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,4825501.4476,61415154757088.0000,7836782.1685,0.8201,0.4347,0.4447


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,21438915.1254,2674848260035854.5000,51718935.2175,-0.0501,1.1914,1.0955
1,17694410.2459,1533732465517348.5000,39162896.5415,0.6069,0.4991,0.4225
2,3039412.5922,18611533926425.3633,4314108.7059,0.9465,0.2758,0.2210
3,3058804.0760,20696674227907.8477,4549359.7602,0.8538,1.0011,1.7582
4,2664809.0358,15845881552538.1152,3980688.5777,0.9014,0.2790,0.2555
Mean,9579270.2151,852746963052014.8750,20745197.7606,0.6517,0.6493,0.7505
Std,8241391.4471,1083723895511025.2500,20551976.3752,0.3700,0.3786,0.5946


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,4003858.4855,44946312599332.8125,6704201.1157,0.8684,0.3831,0.3780


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,21329396.3210,2670414231704124.5000,51676050.8524,-0.0484,1.1910,1.1660
1,17225506.3497,1375653136065606.2500,37089798.2748,0.6474,0.5372,0.4805
2,2644056.2902,14911873529986.6016,3861589.5082,0.9571,0.2395,0.1877
3,2779162.8292,16103005400747.8789,4012855.0187,0.8862,1.1589,2.7715
4,2560008.4284,12659344284799.3828,3557997.2295,0.9212,0.2886,0.2543
Mean,9307626.0437,817948318197053.0000,20039658.1767,0.6727,0.6830,0.9720
Std,8243423.1534,1065737152697696.0000,20404911.6234,0.3765,0.4143,0.9641


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,4333235.0996,50464266565581.8125,7103820.5612,0.8522,0.3921,0.3910


In [9]:
# 准备测试数据
X_test = time_test_data_processed[feature_cols]
y_test = time_test_data_processed[target_column]

# 评估所有模型组合
results_time = model_evaluator.evaluate_ensemble_combinations(X_test, y_test)
pd.DataFrame(results_time).T

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,2224323.1243,30522534510792.8711,5524720.3106,0.9908,0.1419,0.0898


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,3821791.0971,68924402489883.3438,8302072.1805,0.9791,0.3781,0.2210


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,4403129.6800,111158503420705.5625,10543173.3089,0.9664,0.5013,0.5022


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,21300693.8622,2670682209187896.0000,51678643.6469,-0.0485,1.1743,1.0890
1,15356214.0066,984396030609005.0000,31375086.1450,0.7477,0.6219,0.6507
2,2898821.0493,18997611881735.1133,4358624.9990,0.9454,0.2403,0.1867
3,2807112.5457,15523403636233.3672,3939975.0807,0.8903,1.2025,3.2951
4,3136282.9548,21591925295527.5547,4646711.2344,0.8656,0.3200,0.2802
Mean,9099824.8837,742238236122079.3750,19199808.2212,0.6801,0.7118,1.1003
Std,7766828.4151,1034220393175367.2500,19328879.9570,0.3700,0.4095,1.1425


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,2885849.5942,35488030839124.6875,5957183.1296,0.9893,0.2048,0.1485


...

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,21438915.1254,2674848260035854.5000,51718935.2175,-0.0501,1.1914,1.0955
1,17694410.2459,1533732465517348.5000,39162896.5415,0.6069,0.4991,0.4225
2,3039412.5922,18611533926425.3633,4314108.7059,0.9465,0.2758,0.2210
3,3058804.0760,20696674227907.8477,4549359.7602,0.8538,1.0011,1.7582
4,2664809.0358,15845881552538.1152,3980688.5777,0.9014,0.2790,0.2555
Mean,9579270.2151,852746963052014.8750,20745197.7606,0.6517,0.6493,0.7505
Std,8241391.4471,1083723895511025.2500,20551976.3752,0.3700,0.3786,0.5946


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,3982528.5093,79123608312584.6250,8895145.2103,0.9761,0.3700,0.3240


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,21329396.3210,2670414231704124.5000,51676050.8524,-0.0484,1.1910,1.1660
1,17225506.3497,1375653136065606.2500,37089798.2748,0.6474,0.5372,0.4805
2,2644056.2902,14911873529986.6016,3861589.5082,0.9571,0.2395,0.1877
3,2779162.8292,16103005400747.8789,4012855.0187,0.8862,1.1589,2.7715
4,2560008.4284,12659344284799.3828,3557997.2295,0.9212,0.2886,0.2543
Mean,9307626.0437,817948318197053.0000,20039658.1767,0.6727,0.6830,0.9720
Std,8243423.1534,1065737152697696.0000,20404911.6234,0.3765,0.4143,0.9641


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,3315148.9165,53669167031299.7812,7325924.3124,0.9838,0.2953,0.2383


,r2,rmse,mae,mape
model_1,0.990761,5.524720e+06,2.224323e+06,0.089774
model_2,0.979138,8.302072e+06,3.821791e+06,0.220962
model_3,0.966354,1.054317e+07,4.403130e+06,0.502236
model_1_model_2_blend,0.989258,5.957183e+06,2.885850e+06,0.148481
model_1_model_3_blend,0.982204,7.667812e+06,3.183995e+06,0.282141
model_2_model_3_blend,0.976051,8.895145e+06,3.982528e+06,0.323972
model_1_model_2_model_3_blend,0.983755,7.325924e+06,3.315149e+06,0.238342


In [10]:
# 将字典转换为DataFrame后再进行连接
pd.concat([pd.DataFrame(results_country), pd.DataFrame(results_time)]).groupby(level=0).mean().T

,mae,mape,r2,rmse
model_1,3.643182e+06,0.263878,0.901286,6.770290e+06
model_2,3.632081e+06,0.268583,0.940557,7.043639e+06
model_3,4.508181e+06,0.482486,0.895152,9.148109e+06
model_1_model_2_blend,3.543357e+06,0.254861,0.926327,6.393310e+06
model_1_model_3_blend,4.004748e+06,0.363445,0.901165,7.752297e+06
model_2_model_3_blend,3.993193e+06,0.351008,0.922205,7.799673e+06
model_1_model_2_model_3_blend,3.824192e+06,0.314683,0.917977,7.214872e+06


In [11]:
model_evaluator.save_selected_models(
    model_names=['model_1','model_2','model_3'],  # 替换为您要保存的模型名称列表
    save_dir='src/models'  # 可自定义保存路径
)


Transformation Pipeline and Model Successfully Saved
模型 model_1 已保存到 src/models\model_1
Transformation Pipeline and Model Successfully Saved
模型 model_2 已保存到 src/models\model_2
Transformation Pipeline and Model Successfully Saved
模型 model_3 已保存到 src/models\model_3
